In [ ]:
# Section 1: Setup and Data Loading
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import joblib
import os
print("Libraries imported successfully!")
# Load the dataset
try:
    df = pd.read_csv(r'/content/adult 3.csv')
    print("Dataset loaded successfully!")
    print("\nDataset Head:")
    print(df.head())
    print("\nDataset Info:")
    df.info()
    print("\nDataset Shape:", df.shape)
except FileNotFoundError:
    print("Error: 'adult 3.csv' not found. Please upload the file to your Colab environment.")
    exit()

# Section 2: Data Preprocessing
# Replace '?' with NaN across the DataFrame
df.replace('?', np.nan, inplace=True)
print("\nReplaced '?' with NaN values.")

# Drop rows where the target variable 'income' is NaN
df.dropna(subset=['income'], inplace=True)
print("\nDropped rows with missing 'income' values.")

# Define target variable and features
# The target variable is 'income'
X = df.drop('income', axis=1)
y = df['income']

# Identify categorical and numerical columns
# Exclude 'fnlwgt' from numerical features for scaling if it's considered a weight,
# but as per user request, it's an input, so we'll treat it as a numerical feature to be scaled.
# We will also include 'educational-num', 'capital-gain', 'capital-loss' in numerical features.
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(include='object').columns.tolist()

print(f"\nNumerical Features: {numerical_features}")
print(f"Categorical Features: {categorical_features}")

# Create preprocessing pipelines for numerical and categorical features
# Numerical Imputer: Use median for numerical features to handle outliers better than mean
# Categorical Imputer: Use most_frequent for categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # handle_unknown='ignore' is crucial for new categories in prediction
])

# Create a preprocessor using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Encode the target variable 'income'
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print("\nTarget variable 'income' encoded.")
print(f"Original classes: {label_encoder.classes_}")
print(f"Encoded values (0: {label_encoder.classes_[0]}, 1: {label_encoder.classes_[1]})")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
print(f"\nData split into training (X_train shape: {X_train.shape}) and testing (X_test shape: {X_test.shape}) sets.")

# Section 3: Random Forest Classifier Training

# Create the full pipeline including preprocessing and the classifier
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                 ('classifier', RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1))])

# Train the model
print("\nTraining Random Forest Classifier...")
model_pipeline.fit(X_train, y_train)
print("Model training complete!")

# Section 4: Model Evaluation

# Make predictions on the test set
y_pred = model_pipeline.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=label_encoder.classes_)

print(f"\nModel Accuracy: {accuracy:.4f}")
print("\nClassification Report:\n", report)

# Section 5: Model Persistence

# Save the trained model pipeline
model_filename = 'random_forest_salary_model.joblib'
joblib.dump(model_pipeline, model_filename)
print(f"\nModel saved as '{model_filename}'")

# Save the LabelEncoder for inverse transformation later
label_encoder_filename = 'label_encoder.joblib'
joblib.dump(label_encoder, label_encoder_filename)
print(f"LabelEncoder saved as '{label_encoder_filename}'")

# To ensure consistent feature ordering and handling for new predictions,
# we need to save the list of columns after one-hot encoding.
# This requires fitting the preprocessor separately and getting feature names.
# We'll re-fit the preprocessor on the full X to get all column names correctly for prediction.
preprocessor_fitted = preprocessor.fit(X)

# Get feature names after preprocessing
# This part is a bit tricky with ColumnTransformer and OneHotEncoder
# We need to get the feature names from the OneHotEncoder after it's fitted.
# The `get_feature_names_out` method is available on the OneHotEncoder step.

# Get the one-hot encoder step from the preprocessor
onehot_encoder_step = preprocessor_fitted.named_transformers_['cat'].named_steps['onehot']
# Get the original categorical feature names
original_categorical_features = categorical_features
# Get the new feature names out from the one-hot encoder
encoded_categorical_feature_names = onehot_encoder_step.get_feature_names_out(original_categorical_features)

# Combine numerical feature names with the new one-hot encoded feature names
all_feature_names = numerical_features + list(encoded_categorical_feature_names)

# Save the list of all feature names (including one-hot encoded ones)
feature_names_filename = 'model_feature_names.joblib'
joblib.dump(all_feature_names, feature_names_filename)
print(f"Feature names saved as '{feature_names_filename}'")

# Save the preprocessor itself, which contains the fitted scalers and encoders
preprocessor_filename = 'preprocessor.joblib'
joblib.dump(preprocessor_fitted, preprocessor_filename)
print(f"Preprocessor saved as '{preprocessor_filename}'")

print("\nAll necessary model artifacts (model, label encoder, preprocessor, feature names) are saved.")
print("You can now download these .joblib files and use them in your Streamlit application.")

# Example of how to load and use the saved model for a single prediction
# (This part is for testing within Colab, not for the Streamlit app directly)
print("\n--- Testing Saved Model ---")
loaded_model_pipeline = joblib.load(model_filename)
loaded_label_encoder = joblib.load(label_encoder_filename)
loaded_preprocessor = joblib.load(preprocessor_filename)
loaded_feature_names = joblib.load(feature_names_filename)

# Example input based on user's request: age, workclass, fnlwgt, education, occupation, gender, hours-per-week, native-country
# For features not provided by the user (educational-num, marital-status, relationship, race, capital-gain, capital-loss),
# we need to provide default values. We'll use the most frequent/median from the training data.

# Get default values from the original training data (X_train)
# For numerical features not provided by user, use median
default_educational_num = X_train['educational-num'].median()
default_capital_gain = X_train['capital-gain'].median() # or 0 if more appropriate for new entries
default_capital_loss = X_train['capital-loss'].median() # or 0 if more appropriate for new entries

# For categorical features not provided by user, use mode (most frequent)
default_marital_status = X_train['marital-status'].mode()[0]
default_relationship = X_train['relationship'].mode()[0]
default_race = X_train['race'].mode()[0]


sample_input = {
    'age': 35,
    'workclass': 'Private',
    'fnlwgt': 200000,
    'education': 'Bachelors',
    'educational-num': default_educational_num, # Default
    'marital-status': default_marital_status,   # Default
    'occupation': 'Exec-managerial',
    'relationship': default_relationship,       # Default
    'race': default_race,                       # Default
    'gender': 'Male',
    'capital-gain': default_capital_gain,       # Default
    'capital-loss': default_capital_loss,       # Default
    'hours-per-week': 40,
    'native-country': 'United-States'
}

# Create a DataFrame from the sample input
sample_df = pd.DataFrame([sample_input])

# Ensure the order of columns in sample_df matches the original training DataFrame X
# This is crucial for the preprocessor to work correctly.
# We'll reorder sample_df columns based on X.columns
sample_df = sample_df[X.columns]

print("\nSample input for prediction:")
print(sample_df)

# Make prediction
predicted_label_encoded = loaded_model_pipeline.predict(sample_df)
predicted_income = loaded_label_encoder.inverse_transform(predicted_label_encoded)

print(f"\nPredicted Income for sample input: {predicted_income[0]}")

Libraries imported successfully!
Dataset loaded successfully!

Dataset Head:
   age  workclass  fnlwgt     education  educational-num      marital-status  \
0   25    Private  226802          11th                7       Never-married   
1   38    Private   89814       HS-grad                9  Married-civ-spouse   
2   28  Local-gov  336951    Assoc-acdm               12  Married-civ-spouse   
3   44    Private  160323  Some-college               10  Married-civ-spouse   
4   18          ?  103497  Some-college               10       Never-married   

          occupation relationship   race  gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black    Male             0             0   
1    Farming-fishing      Husband  White    Male             0             0   
2    Protective-serv      Husband  White    Male             0             0   
3  Machine-op-inspct      Husband  Black    Male          7688             0   
4                  ?    Own-child  White  

# New Section